# [주의] 로컬 쥬피터에서 실행 후 데이터 c 드라이브에서 바로 추출하기
### 1. errorLogPath(제대로 변환 안 된 파일 확인), path(5초 input data 있는 곳), folder(파일저장) 확인하기
### 2. 실행 후 쥬피터에서 errorlog에 쓰여진 파일 있으면 파일 추출하지 말고 지수한테 말하기

In [ ]:
import librosa
import os
import numpy as np

# Load the audio file using librosa
errorLogPath = "testpitch_error_log.txt"
path = './test/' # path = './파일 있는 경로/'
file_list = os.listdir(path)
file_list_wav = [file for file in file_list if file.endswith('.wav')]
i=1 ; folder_num = 0
error_log = []

for filename in file_list_wav :
    audio_data, sr = librosa.load(path + filename, sr = 16000)

# Extract the fundamental frequency using the Yin algorithm
    fundamental_frequency, voiced_flag, voiced_probs = librosa.pyin(audio_data, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    times = librosa.times_like(fundamental_frequency, sr=sr)
    data = fundamental_frequency


    NAN = False ; B_num = 0 ; NUN_array = [] ; data2 = []
    for i in range (len(data)) :

        if NAN == False and not np.isnan(data[i]): #not_nun
            B_index = i
            B_num = data[i]

        elif np.isnan(data[i]) and NAN == False :  #nun_starting point

            if i == len(data) -1 : # only 1 nan
                data[i] = B_num / 2

            NAN = True
            B_nan = i

        elif not np.isnan(data[i]) and NAN == True : #nun_ending point
            avg = (B_num + data[i]) / 2
            for j in range (B_nan, i) :
                data[j] = avg
            NAN = False
            B_index = i ; B_num = data[i]

        elif np.isnan(data[i]) and i == len(data) - 1 : #nun_ending with data_ending
            avg = B_num / 2
            for j in range (B_nan, i+1) :
                data[j] = avg



#     data = {'time': times,  'fundamental_frequency': fundamental_frequency}

    folder = './PITCH_test/'
    file_path = folder  + str(folder_num)
    folder_num += 1
    np.save(file_path, data)


    if True in (np.isnan(np.load(file_path + '.npy'))) :
        error_log.append(filename)


# 파일 이름 넣기
with open(errorLogPath, "w") as f:
    for filename in error_log:
        f.write(f"{filename}\n")